In [ ]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt

from red_utils import split_rel_results

In [ ]:
res_df = 'res_df.2458099.43869.ee.pkl'

In [ ]:
with open(res_df.rsplit('.', 1)[0] + '.md.pkl', 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(res_df)
df

In [ ]:
# check results for a given frequency & time integration
test_freq = 300
test_tint = 35

resx = df.loc[(test_freq, test_tint)][5:].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'])
test_amps = np.abs(test_gains)
test_phases = np.angle(test_gains)
mean_test_amps = np.mean(test_amps)
mean_test_phases = np.mean(test_phases)

print('Gain amplitude solutions for frequency channel {} and time integration {} are:\n{}\n'\
      .format(test_freq, test_tint, test_amps))
print('Gain phase solutions are:\n{}\n'\
      .format(test_phases))
print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
def plot_res(df, ycol, group_by, ylabel, logy=False):
    """"""
    idx_dict = {k:i for i, k in enumerate(df.index.values)}
    
    x1 = [idx_dict[i] for i in df[df[group_by]][ycol].index.values]
    y1 = df[df[group_by]][ycol].values

    x2 = [idx_dict[i] for i in df[~df[group_by]][ycol].index.values]
    y2 = df[~df[group_by]][ycol].values
    
    if logy:
        y1 = np.log(y1)
        y2 = np.log(y2)

    fig, ax = plt.subplots(figsize=(12,8))
    ax.scatter(x1, y1, s=4, alpha=0.5, label=group_by)
    ax.scatter(x2, y2, s=4, color='orange', zorder=1, label='~'+group_by)
    ax.set_ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
ax = df['nit'].plot(figsize=(12,8))
ax.set_ylabel('Number of iterations')
plt.show()

In [ ]:
plot_res(df, 'nit', 'success', 'Log number of iterations', logy=True)

In [ ]:
print('Max number of iterations for unsuccesful minimizations: {}\n'.\
      format(np.max(df['nit'][df['success'] == False].values)))

print('Max number of iterations for succesful minimizations: {}'.\
      format(np.max(df['nit'][df['success'] == True].values)))

### Log-likelihood

In [ ]:
ax = df['fun'].plot(figsize=(12,8), ylim=[-0.1, 1])
ax.set_ylabel('Log-likelihood')
plt.show()

In [ ]:
plot_res(df, 'fun', 'success', 'Log log-likelihood', logy=True)

In [ ]:
print('Max log-likelihood for unsuccesful minimizations: {}\n'.\
      format(round(np.max(df['fun'][df['success'] == False].values), 3)))

print('Max log-likelihood for succesful minimizations: {}'.\
      format(round(np.max(df['fun'][df['success'] == True].values), 3)))